In [1]:
from pynq import Overlay
Overlay("base.bit").download()

Setup code:
imports Pynq board functions for use
declares LED pins
Declares PMOD Control for the Grove IMU

In [2]:
from time import sleep
import math
from pynq.board import Button
from pynq.iop import Arduino_IO
from pynq.iop import ARDUINO
from pynq.iop import PMODA
from pynq.iop import PMOD_GROVE_G4
from pynq.iop import ARDUINO_GROVE_I2C
from pynq.iop import Grove_IMU
sense = Grove_IMU(PMODA, PMOD_GROVE_G4)
led = Arduino_IO(ARDUINO,13,'out')
led1 = Arduino_IO(ARDUINO,12,'out')
led2 = Arduino_IO(ARDUINO,11,'out')
print('start')
btns = [Button(index) for index in range(4)]
led.write(0)
led1.write(0)
led2.write(0)

start


## Define Fuctions to determine heading and account for small degrees of compass tilt

getAngleDeg: returns angle between x axis and magnetic north in degrees

get_tilt_heading: returns angle betwwen the x axis and magnetic north in degrees, adjusted for small amounts of tilt in the compass

In [3]:
def getAngleDeg(x,y,mod):
    h=math.sqrt(x*x + y*y)
    rawAng=math.asin(y/h) / (2*math.pi) *360
    if (x>=0 and y>=0):
        adjAng = rawAng
    elif (x<0 and y>=0):
        adjAng = 180-rawAng
    elif (x>=0 and y<0):
        adjAng = 360+rawAng
    elif (x<0 and y<0):
        adjAng = 180-rawAng
    else:
        adjAng = rawAng
        print('error in angle calcs')
        return 0;
    finalAng= adjAng + mod;
    return finalAng
    
def get_tilt_heading(data,altdata):
    [ax, ay, _] = altdata[0:3]
    [mx, my, mz] = data[0:3]
    try:
        pitch = math.asin(-ax)
        roll = math.asin(ay / math.cos(pitch))
    except ZeroDivisionError:
        raise RuntimeError("Value out of range or device not connected")

    xh = mx * math.cos(pitch) + mz * math.sin(pitch)
    yh = mx * math.sin(roll) * math.sin(pitch) + \
    my * math.cos(roll) - mz * math.sin(roll) * math.cos(pitch)
    _ = -mx * math.cos(roll) * math.sin(pitch) + \
    my * math.sin(roll) + mz * math.cos(roll) * math.cos(pitch)
    tilt_heading = 180 * math.atan2(yh, xh) / math.pi
    if yh < 0:
        tilt_heading += 360
    return float("{0:.2f}".format(tilt_heading))

central Loop:
will light up the central LED only When pointed at Magnetic North
press and hold on board Button 3 to terminate

In [7]:
print('start')
sense.reset()
data=sense.get_compass()
heading=sense.get_heading()
altdata=sense.get_accl()
print(data)
print(heading)
offset=heading;
while (btns[3].read()==0):
    sense.reset()
    data=sense.get_compass()
    heading=get_tilt_heading(data, altdata)
    setheading= (heading - offset+360)%360 
    #print(data)
    #print(getAngleDeg(data[0],data[1],0))
    print(setheading)
    if (setheading < 5 or setheading >355):
        led.write(1)
        led1.write(0)
        led2.write(0)
    elif (setheading < 30 and setheading > 5):
        led1.write(0)
        led.write(0)
        led2.write(1)
    elif (setheading < 355 and setheading > 330):
        led2.write(0)
        led.write(0)
        led1.write(1)
    else:
        led.write(0)
    #print(altitude(data[0],data[1],data[2]))
    sleep(1)
print('done')
led.write(0)
led1.write(0)
led2.write(0)

start
[15.23, 12.01, 9.38]
38.26
359.93
0.05000000000001137
355.21
354.7
358.31
351.52
354.46
358.52
359.37
5.639999999999986
5.019999999999982
358.32
357.44
357.21
356.47
356.93
done
